In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

base_path = "../build/03_sine_waves"

values = pd.read_csv(f"{base_path}/output.csv")

values['time'] = values.index * (1/44100)



values[-5000:].plot(x='time', figsize=(30, 5))


In [ ]:
fft = pd.read_csv(f"{base_path}/mags.csv", names=['Frequency Bins'])
fft['freq'] = fft.index * (44100)/(1024 * 1024)
fft[:22000].plot(x='freq', y='Frequency Bins', figsize=(10,5))